In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
INPUT_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Indirect Lending\Adhoc_Pricing_Disparity_20250822\output\output.parquet")

In [ ]:
df = pd.read_parquet(INPUT_PATH)

In [ ]:
df

In [ ]:
from deltalake import DeltaTable

In [ ]:
ACCOUNT_DATA = Path(r"C:\Users\w322800\Documents\lakehouse\silver\account")

In [ ]:
active_accounts = DeltaTable(ACCOUNT_DATA).to_pandas()

In [ ]:
active_accounts

In [ ]:
active_accounts = active_accounts[(active_accounts['Category'].isin(['Indirect'])) | (active_accounts['currmiaccttypcd']).isin(['CM15','CM16'])].copy()

In [ ]:
df.info()

In [ ]:
active_accounts_slice = active_accounts[[
    'acctnbr',
    'ownersortname',
    'contractdate',
    'curracctstatcd',
    'noteopenamt',
    'notebal',
    'noteintrate'
]].copy()

In [ ]:
active_accounts_slice = active_accounts_slice.rename(columns={
    'acctnbr': 'Account Number',
    'contractdate': 'Loan Origination Date',
    'ownersortname': 'Applicant Last Name', # Needs splitting
    'noteopenamt': 'Amount Financed',
    'notebal': 'Current Balance',
    'noteintrate': 'Contract Rate',
}).copy()

In [ ]:
active_accounts_slice.info()

In [ ]:
merged_df = pd.merge(df, active_accounts_slice, on='Account Number', how='outer', suffixes=('_df','_active'), indicator=True)

In [ ]:
merged_df

In [ ]:
merged_df.info()

In [ ]:
import numpy as np

In [ ]:
merged_df['Status'] = np.where(merged_df['_merge'] == 'left_only', 'Closed/Charged Off','Active')